# Standard survey 

The same total count is used in...

## RAG summary: Introduction 

"At your school, a total of [number] pupils took part in the #BeeWell survey. This section shows how the answers of pupils at your school compare with pupils from other schools in Northern Devon."

## RAG summary: Understanding topic scores

"Please note that although a total of [number] pupils from your school took part, the RAG ratings given in this section are based only on responses from pupils who completed all the questions within each topic."

## Who took part?

""There were [number] pupils at your school who took part in the #BeeWell survey. This section describes the sample of pupils who completed the survey."

# Standard survey 

## For every topic, "Comparison with other schools"

"Your school had [number] complete responses. Across Northern Devon, there were [number]] complete responses from [number] schools. The average score for the pupils at your school, compared to other schools in Northern Devon, was:

[RAG rating for subgroup]"


* All pupils 
* By year group (Year 8, Year 10)
* By gender (Boys, Girls)
* By FSM status (non-FSM, FSM)
* By SEN status (non-SEN, SEN)


# Introduction for both STANDARD and SYMBOL (only needs to be generated once)

## About: Who took part in the #BeeWell survey in North Devon?


# Standard
Check these values:

"This year, 1298 pupils in Years 8 and 10 at five secondary schools from across North Devon and Torridge completed the standard version of the #BeeWell survey. 

A symbol version of the survey was also completed by 101 pupils in Years 7 to 11 from two non-mainstream schools in Northern Devon."


# Symbol



In [3]:
import pandas as pd
import os

totals_df = pd.read_csv(
    "../data/real/all_schools_overall_counts.csv",
)

# Filter for rows where all categories are "All"
all_pupils_totals = totals_df[
    (totals_df["year_group_lab"] == "All")
    & (totals_df["fsm_lab"] == "All")
    & (totals_df["sen_lab"] == "All")
    & (totals_df["gender_lab"] == "All")
]

# Calculate the total counts for each survey type
standard_total = all_pupils_totals[all_pupils_totals["survey_type"] == "standard"]["count"].dropna().sum()
symbol_total = all_pupils_totals[all_pupils_totals["survey_type"] == "symbol"]["count"].dropna().sum()

# Count unique schools for each survey type
standard_schools = all_pupils_totals[all_pupils_totals["survey_type"] == "standard"]["school_lab"].nunique()
symbol_schools = all_pupils_totals[all_pupils_totals["survey_type"] == "symbol"]["school_lab"].nunique()

# Preparing the output statement
output_statement = (
    f"This year, {int(standard_total)} pupils in Years 8 and 10 at {standard_schools} secondary schools from across "
    "North Devon and Torridge completed the standard version of the #BeeWell survey. A symbol version of the survey "
    f"was also completed by {int(symbol_total)} pupils in Years 7 to 11 from {symbol_schools} non-mainstream schools "
    "in Northern Devon."
)

print(output_statement)
print(all_pupils_totals)

# Save the result to a CSV file


# Filter for rows where all categories are "All"
all_pupils_totals = totals_df[
    (totals_df["year_group_lab"] == "All")
    & (totals_df["fsm_lab"] == "All")
    & (totals_df["sen_lab"] == "All")
    & (totals_df["gender_lab"] == "All")
]

# Select only one row per school with the total count
school_totals = all_pupils_totals.groupby("school_lab", as_index=False)["count"].sum()

# Display the result
print(school_totals)

# Save the result to a CSV file
school_totals.to_csv("outputs/all_pupils_totals_by_school.csv", index=False)


This year, 1301 pupils in Years 8 and 10 at 5 secondary schools from across North Devon and Torridge completed the standard version of the #BeeWell survey. A symbol version of the survey was also completed by 101 pupils in Years 7 to 11 from 2 non-mainstream schools in Northern Devon.
    count                    school_lab year_group_lab fsm_lab sen_lab  \
0     292      BRAUNTON SCHOOL AND C.C.            All     All     All   
9     234       GREAT TORRINGTON SCHOOL            All     All     All   
18    139  HOLSWORTHY COMMUNITY COLLEGE            All     All     All   
27    394      PILTON COMMUNITY COLLEGE            All     All     All   
36    242        THE ILFRACOMBE ACADEMY            All     All     All   
45     40              PATHFIELD SCHOOL            All     All     All   
54     61  THE LAMPARD COMMUNITY SCHOOL            All     All     All   

   gender_lab survey_type  Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  \
0         All    standard         NaN      

In [4]:

# Load your dataset
# Load your dataset
data = pd.read_csv("../data/real/standard_area_aggregate_scores_rag.csv")

# Replace specific values in 'variable_lab' for clarity
data["variable_lab"] = data["variable_lab"].replace(
    {
        "Taking to staff about feelings": "Talking to staff about feelings",
        "Taking at home about feelings": "Talking at home about feelings",
        "Taking to peers about feelings": "Talking to peers about feelings",
        "Talking about feelings": "General talking about feelings",  # Temporary label for confirmation before filtering out
    }
)

# Update 'group' column and relevant 'variable_lab' and 'description' values for specific 'variable' entries
data.loc[data["variable"] == "staff_talk_score", ["variable_lab", "description", "group"]] = [
    "Talking to staff about feelings",
    "How positively/negatively young people feel about talking with staff about feeling down",
    "staff_talk",
]

data.loc[data["variable"] == "peer_talk_score", ["variable_lab", "description", "group"]] = [
    "Talking to peers about feelings",
    "How positively/negatively young people feel about talking with peers about feeling down",
    "peer_talk",
]

data.loc[data["variable"] == "home_talk_score", ["variable_lab", "description", "group"]] = [
    "Talking at home about feelings",
    "How positively/negatively young people feel about talking at home about feeling down",
    "home_talk",
]

# Remove rows where 'variable' is 'talk_score' to avoid redundancy
data = data[data["variable"] != "talk_score"]


# Group data by school, topic, and subgroups for processing
grouped_data = data.groupby(["school_lab", "variable", "year_group_lab", "gender_lab", "fsm_lab", "sen_lab"])

# Function to generate readable subgroup name from attributes
def generate_subgroup_name(year_group, gender, fsm, sen):
    # Exclude 'All' and join remaining parts with spaces; use 'All' if all are 'All'
    parts = [year_group, gender, fsm, sen]
    name = " ".join([part for part in parts if part != "All"])
    return name if name else "All"


# Loop over each school, topic, and subgroup
for (school_name, topic, year_group, gender, fsm, sen), group in grouped_data:
    # Define output directory and ensure it exists
    subgroup_name = generate_subgroup_name(year_group, gender, fsm, sen)
    output_dir = f"outputs/{school_name}/responses_totals/{subgroup_name}"
    os.makedirs(output_dir, exist_ok=True)

    # Define the file path for each markdown file using `variable` (topic) for filename
    file_path = f"{output_dir}/{topic}.md"

    # Prepare data for the markdown content
    x = int(group["count"].values[0])  # Number of responses in the school for this topic
    y = int(group["total_pupils"].values[0])  # Total regional responses for this topic

    # Calculate total unique schools for this topic/subgroup combination across Northern Devon
    z = data[
        (data["variable"] == topic)
        & (data["year_group_lab"] == year_group)
        & (data["gender_lab"] == gender)
        & (data["fsm_lab"] == fsm)
        & (data["sen_lab"] == sen)
    ]["school_lab"].nunique()

    # Get RAG rating for the school's subgroup and replace with full description
    rag = (
        group["rag"]
        .replace(
            {
                "average": "Average",
                "above": "Above average",
                "below": "Below average",
            }
        )
        .values[0]
    )

    # Generate the markdown content
    markdown_content = f"""# {school_name}

## {topic}

### {subgroup_name}

Your school had {x} complete responses. Across Northern Devon, there were {y} complete responses from {z} schools. 
The average score for the pupils at your school, compared to other schools in Northern Devon, was:

**{rag}**
"""

    # Write the content to the markdown file
    with open(file_path, "w") as file:
        file.write(markdown_content)

print("Markdown files generated in 'outputs/{school}/responses_totals/{subgroup_name}/{topic}.md'")


Markdown files generated in 'outputs/{school}/responses_totals/{subgroup_name}/{topic}.md'


# Symbol survey




## Who took part?

"There were 30 pupils at your school who took part in the #BeeWell survey. This page describes the sample of pupils who completed the survey."